# Introduction to Nested-Pandas

This notebook explores the Nested-Pandas API, showing the basics of nesting data and touring the various ways of working with nested data.

## Exploring the NestedFrame Interface

In [20]:
import nested_pandas as npd
import light_curve as licu
import pandas as pd
import numpy as np

from nested_pandas.utils import count_nested

In [ ]:
# Load an example dataset
from nested_pandas.datasets import generate_data
ndf = generate_data(50,100, seed=1)
ndf

a         b                                             nested
0   0.417022  0.038734  [{t: 6.532898, flux: 77.388964, band: 'r'}; …]...
1   0.720324  1.357671  [{t: 10.541162, flux: 82.03493, band: 'r'}; …]...
..       ...       ...                                                ...
48  0.287775  0.005741  [{t: 11.19434, flux: 11.946375, band: 'r'}; …]...
49  0.130029  1.234290  [{t: 0.25112, flux: 94.006251, band: 'g'}; …] ...

[50 rows x 3 columns]

## Approaches for Nesting Data

## Performance Comparisons to Pandas

### Taking Native Pandas Code

In [17]:
%%timeit

# Read data
object_df = pd.read_parquet("data/objects.parquet")
source_df = pd.read_parquet("data/ztf_sources.parquet")

# Filter on object
filtered_object = object_df.query("ra > 10.0")
#sync object to source --removes any index values of source not found in object
filtered_source = filtered_object[[]].join(source_df, how="left")

# Count number of observations per photometric band and add it to the object table
band_counts = source_df.groupby(level=0).apply(lambda x:
                                               x[["band"]].value_counts().reset_index()).pivot_table(values="count",
                                                                                                     index="index",
                                                                                                     columns="band",
                                                                                                     aggfunc="sum")
filtered_object = filtered_object.join(band_counts[["g","r"]])

# Filter on our nobs
filtered_object = filtered_object.query("g > 520")
filtered_source = filtered_object[[]].join(source_df, how="left")

# Calculate Amplitude
amplitude = licu.Amplitude()
filtered_source.groupby(level=0).apply(lambda x: amplitude(np.array(x.mjd), np.array(x.flux)))

530 ms ± 7.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Simplified and Faster with Nested-Pandas

In [21]:
%%timeit

#Read in parquet data
#nesting sources into objects
nf = npd.read_parquet(data="data/objects.parquet",
                  to_pack={"ztf_sources": "data/ztf_sources.parquet"})

# Filter on object
nf = nf.query("ra > 10.0")

# Count number of observations per photometric band and add it as a column
nf = count_nested(nf, "ztf_sources", by="band", join=True)

# Filter on our nobs
nf = nf.query("n_ztf_sources_g > 520")

# Calculate Amplitude
amplitude = licu.Amplitude()
nf.reduce(amplitude, "ztf_sources.mjd", "ztf_sources.flux")

251 ms ± 20.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Hands-on Scientific Example: Variability Analysis

### Load and Nest ZTF Timeseries Data

### Perform Initial Filtering

### Calculate Periodograms for all Lightcurves

### Visualizing Results

### Using Results to Modify our NestedFrame